In [1]:
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import math
import scipy as sp
from numba import cuda, jit
import numba
import threading
import time
from collections import defaultdict
import json

### Helper functions

In [2]:
def my_binary(n,p):
    bin_exp=np.zeros(p).astype(int)
    for i in range(p):
        bin_exp[i]=n % 2
        n=n//2
    return bin_exp

def all_sets(p, k):
    if k==0:
        return [[]]
    if p==1:
        return [[],[0]]
    prev = all_sets(p-1, k)
    ans = []
    ans.extend(prev)
    for x in prev:
        if len(x)<k:
            ans.append(x+[p-1])
    ans.sort(key = lambda x:(len(x),x))
    return ans

def magic_matrix(p,k):
    my_mat=np.zeros((1,p)).astype(int)
    comb_list = all_sets(p,k)
    for i in range(1,len(comb_list)):
        my_binary_vec=np.zeros(p).astype(int)
        for j in range(len(comb_list[i])):
            my_binary_vec[comb_list[i][j]]=1
        my_mat=np.concatenate((my_mat,np.array([my_binary_vec])),axis=0)
    #my_mat[0]=1
    return  my_mat

#@jit(nopython=True)
def magic_matrix_alt(p,k):
    my_mat=np.zeros((1,p)).astype(int)
    for i in range(1,2**p):
        my_binary_vec=my_binary(i,p)
        if my_binary_vec.sum() <= k :
            my_mat=np.concatenate((my_mat,np.array([my_binary_vec])),axis=0)
        def bin(x):
            return (2**p)*np.sum(x)-np.dot(x,2**np.array(range(p-1,-1,-1)))
        my_mat=my_mat[np.apply_along_axis(bin, 1, my_mat).argsort(),:]
    my_mat[0]=1
    return  my_mat

In [40]:
#@ray.remote(num_gpus=1)
def single_round_inf(num_p, num_k, num_samples,dosage_grid,num_trials):
    results = defaultdict(list)
    X_gen=magic_matrix(num_p,num_k)
    X_gen_cuda=cp.asarray(X_gen)
    beta = cp.random.uniform(low = -1, high = 1, size = len(X_gen))
    beta = cp.reshape(beta, (len(X_gen),1))
    K = 1*len(beta)
    mean = np.zeros((num_samples,1))
    X = cp.zeros((num_samples, num_p))
    sigma = 1


    @cp.fuse
    def X_worker(Y):
        return 1 - 2*(Y % 2)


    for i in range(len(dosage_grid)):
        if i%100==0:
            print(num_p," ",num_k," ",i)
        # dosage_vec
        dosage_vec = cp.random.uniform(low = 0.5-dosage_grid[i], high = 0.5+dosage_grid[i], size = num_p)
        dosage_vec[0] = cp.random.choice([0.5 - dosage_grid[i],0.5 + dosage_grid[i]], size = 1)
        dosage_vec_cuda = cp.random.permutation(dosage_vec)

        for _ in range(num_trials):
            # The Real X matrix
            X=cp.transpose(X_worker(cp.matmul(X_gen_cuda,cp.random.uniform(size = (num_p, num_samples))>dosage_vec_cuda[:,cp.newaxis])))
            XTX = cp.matmul(cp.transpose(X), X)
            evals = cp.linalg.eigvalsh(XTX)
            #print(evals.shape)

                # Y vector

            #scale = cp.sqrt(sigma**2*(cp.sum(X[:,1:num_p+1], axis = 1)+num_p)/2)
            Y = cp.matmul(X,beta) + cp.random.normal(scale = sigma, size = (num_samples,1))
            ols_bound = sigma**2*cp.sum(1/evals)

            if ols_bound<K:
                beta_hat = cp.matmul(cp.linalg.inv(XTX),cp.matmul(cp.transpose(X),Y))
            else:
                beta_hat = cp.zeros((len(beta),1))
            results[dosage_grid[i]].append((cp.linalg.norm(beta-beta_hat)**2).item())
    return results




In [15]:
#@ray.remote(num_gpus=1)
def single_round_inf_constrained(num_p, num_k, L, num_samples,dosage_grid,num_trials):
    results = defaultdict(list)
    X_gen=magic_matrix(num_p,num_k)
    X_gen_cuda=cp.asarray(X_gen)
    beta = cp.random.uniform(low = -1, high = 1, size = len(X_gen))
    beta = cp.reshape(beta, (len(X_gen),1))
    K = 1*len(beta)
    mean = np.zeros((num_samples,1))
    X = cp.zeros((num_samples, num_p))
    sigma = 1

    def isValid(d):
        return cp.all(d>=0) and cp.all(d<=1)

    @cp.fuse
    def X_worker(Y):
        return 1 - 2*(Y % 2)


    for i in range(len(dosage_grid)):
        if i%100==0:
            print(num_p," ",num_k," ",i)
        # dosage_vec

        dosage_vec = cp.random.uniform(low = L/num_p-dosage_grid[i], high = L/num_p+dosage_grid[i], size = num_p)
        dosage_vec[0] = cp.random.choice([L/num_p - dosage_grid[i],L/num_p + dosage_grid[i]], size = 1)
        while not isValid(dosage_vec) or cp.sum(dosage_vec)>L:
            dosage_vec = cp.random.uniform(low = L/num_p-dosage_grid[i], high = L/num_p+dosage_grid[i], size = num_p)
            dosage_vec[0] = cp.random.choice([L/num_p - dosage_grid[i],L/num_p + dosage_grid[i]], size = 1)
        dosage_vec_cuda = cp.random.permutation(dosage_vec)

        for j in range(num_trials):
            #print(j)
            # The Real X matrix
            X=cp.transpose(X_worker(cp.matmul(X_gen_cuda,cp.random.uniform(size = (num_p, num_samples))>dosage_vec_cuda[:,cp.newaxis])))
            XTX = cp.matmul(cp.transpose(X), X)
            evals = cp.linalg.eigvalsh(XTX)
            #print(evals.shape)

                # Y vector

            #scale = cp.sqrt(sigma**2*(cp.sum(X[:,1:num_p+1], axis = 1)+num_p)/2)
            Y = cp.matmul(X,beta) + cp.random.normal(scale = sigma, size = (num_samples,1))
            ols_bound = sigma**2*cp.sum(1/evals)

            if ols_bound<K:
                beta_hat = cp.matmul(cp.linalg.inv(XTX),cp.matmul(cp.transpose(X),Y))
            else:
                beta_hat = cp.zeros((len(beta),1))
            results[dosage_grid[i]].append((cp.linalg.norm(beta-beta_hat)**2).item())
    return results




In [41]:
num_trials = 20
dosage_grid = np.repeat(np.arange(0, .41, .02), 100)
results_single_round_variable_20_2_1000 = single_round_inf(num_p = 20, num_k = 2, num_samples = 1000,dosage_grid = dosage_grid,num_trials = num_trials)
'''
results_single_round_constrained_10_2_200 = single_round_inf_constrained(num_p = 10, num_k = 2, L = 2,num_samples = 1000,dosage_grid = dosage_grid,num_trials = num_trials)
'''
'''
results_single_round_constrained_9_2_1000 = single_round_inf_constrained(num_p = 9, num_k = 2, L=2, num_samples = 1000,dosage_grid = dosage_grid,num_trials = num_trials)
'''
'''
results_single_round_variable_30_2_2000 = single_round_inf(num_p = 30, num_k = 2, num_samples = 2000,dosage_grid = dosage_grid,num_trials = num_trials)
'''
#results_single_round_variable_10_1_100 = single_round_inf(num_p = 10, num_k = 1, num_samples = 100,dosage_grid = dosage_grid,num_trials = num_trials)
'''
results_single_round_variable_20_1_100 = single_round_inf(num_p = 20, num_k = 1, num_samples = 100,dosage_grid = dosage_grid,num_trials = num_trials)
results_single_round_variable_30_1_100 = single_round_inf(num_p = 30, num_k = 1, num_samples = 100,dosage_grid = dosage_grid,num_trials = num_trials)
'''


20   2   0
20   2   100
20   2   200
20   2   300
20   2   400
20   2   500
20   2   600
20   2   700
20   2   800
20   2   900
20   2   1000
20   2   1100
20   2   1200
20   2   1300
20   2   1400
20   2   1500
20   2   1600
20   2   1700
20   2   1800
20   2   1900
20   2   2000


'\nresults_single_round_variable_20_1_100 = single_round_inf(num_p = 20, num_k = 1, num_samples = 100,dosage_grid = dosage_grid,num_trials = num_trials)\nresults_single_round_variable_30_1_100 = single_round_inf(num_p = 30, num_k = 1, num_samples = 100,dosage_grid = dosage_grid,num_trials = num_trials)\n'

In [42]:
import json

with open('results_single_round_variable_20_2_1000.txt', 'w') as file:
    file.write(json.dumps(results_single_round_variable_20_2_1000))

In [39]:
import json

with open('results_single_round_constrained_10_2_200.txt', 'w') as file:
    file.write(json.dumps(results_single_round_constrained_10_2_200))

with open('results_single_round_constrained_9_2_1000.txt', 'w') as file:
    file.write(json.dumps(results_single_round_constrained_9_2_1000))
'''
with open('results_single_round_variable_30_2_2000.txt', 'w') as file:
    file.write(json.dumps(results_single_round_variable_30_2_2000))

#with open('results_single_round_variable_10_1_100.txt', 'w') as file:
    #file.write(json.dumps(results_single_round_variable_10_1_100))
    '''
'''
with open('results_single_round_variable_20_1_100.txt', 'w') as file:
    file.write(json.dumps(results_single_round_variable_20_1_100))
with open('results_single_round_variable_30_1_100.txt', 'w') as file:
    file.write(json.dumps(results_single_round_variable_30_1_100))
'''

"\nwith open('results_single_round_variable_20_1_100.txt', 'w') as file:\n    file.write(json.dumps(results_single_round_variable_20_1_100))\nwith open('results_single_round_variable_30_1_100.txt', 'w') as file:\n    file.write(json.dumps(results_single_round_variable_30_1_100))\n"

In [26]:
#@ray.remote(num_gpus=1)
def single_round_uniform(num_p, num_k, num_samples,uniform_grid,num_trials):
    results = defaultdict(list)
    X_gen=magic_matrix(num_p,num_k)
    X_gen_cuda=cp.asarray(X_gen)
    beta = cp.random.uniform(low = -1, high = 1, size = len(X_gen))
    beta = cp.reshape(beta, (len(X_gen),1))
    K = 1*len(beta)
    mean = np.zeros((num_samples,1))
    X = cp.zeros((num_samples, num_p))
    sigma = 1

    @cp.fuse
    def X_worker(Y):
        return 1 - 2*(Y % 2)

    for i in range(len(uniform_grid)):
        print(i)
        # dosage_vec
        dosage_vec = [uniform_grid[i]]*num_p
        dosage_vec_cuda = cp.asarray(dosage_vec)

        for j in range(num_trials):
            if j%100==0:
                print(num_p, num_k, j)
            # The Real X matrix
            X=cp.transpose(X_worker(cp.matmul(X_gen_cuda,cp.random.uniform(size = (num_p, num_samples))>dosage_vec_cuda[:,cp.newaxis])))
            XTX = cp.matmul(cp.transpose(X), X)
            evals = cp.linalg.eigvalsh(XTX)


                # Y vector


            Y = cp.matmul(X,beta) + cp.random.normal(scale = sigma, size = (num_samples,1))
            ols_bound = sigma**2*cp.sum(1/evals)


            if ols_bound<K:
                beta_hat = cp.matmul(cp.linalg.inv(XTX),cp.matmul(cp.transpose(X),Y))
            else:
                beta_hat = cp.zeros((len(beta),1))
            results[uniform_grid[i]].append((cp.linalg.norm(beta-beta_hat)**2).item())
    return results



In [12]:
uniform_grid = np.arange(.4, .61, .01)
results_single_round_variable_uniform_10_2_6000 = single_round_uniform(num_p = 10, num_k = 2, num_samples = 6000,uniform_grid = uniform_grid,num_trials = 500)
with open('results_single_round_variable_uniform_10_2_6000.txt', 'w') as file:
    file.write(json.dumps(results_single_round_variable_uniform_10_2_6000))
results_single_round_variable_uniform_20_2_6000 = single_round_uniform(num_p = 20, num_k = 2, num_samples = 6000,uniform_grid = uniform_grid,num_trials = 500)
with open('results_single_round_variable_uniform_20_2_6000.txt', 'w') as file:
    file.write(json.dumps(results_single_round_variable_uniform_20_2_6000))
results_single_round_variable_uniform_30_2_6000 = single_round_uniform(num_p = 30, num_k = 2, num_samples = 6000,uniform_grid = uniform_grid,num_trials = 500)
with open('results_single_round_variable_uniform_30_2_6000.txt', 'w') as file:
    file.write(json.dumps(results_single_round_variable_uniform_30_2_6000))

0
10 2 0
10 2 100
10 2 200
10 2 300
10 2 400
1
10 2 0
10 2 100
10 2 200
10 2 300
10 2 400
2
10 2 0
10 2 100
10 2 200
10 2 300
10 2 400
3
10 2 0
10 2 100
10 2 200
10 2 300
10 2 400
4
10 2 0
10 2 100
10 2 200
10 2 300
10 2 400
5
10 2 0
10 2 100
10 2 200
10 2 300
10 2 400
6
10 2 0
10 2 100
10 2 200
10 2 300
10 2 400
7
10 2 0
10 2 100
10 2 200
10 2 300
10 2 400
8
10 2 0
10 2 100
10 2 200
10 2 300
10 2 400
9
10 2 0
10 2 100
10 2 200
10 2 300
10 2 400
10
10 2 0
10 2 100
10 2 200
10 2 300
10 2 400
11
10 2 0
10 2 100
10 2 200
10 2 300
10 2 400
12
10 2 0
10 2 100
10 2 200
10 2 300
10 2 400
13
10 2 0
10 2 100
10 2 200
10 2 300
10 2 400
14
10 2 0
10 2 100
10 2 200
10 2 300
10 2 400
15
10 2 0
10 2 100
10 2 200
10 2 300
10 2 400
16
10 2 0
10 2 100
10 2 200
10 2 300
10 2 400
17
10 2 0
10 2 100
10 2 200
10 2 300
10 2 400
18
10 2 0
10 2 100
10 2 200
10 2 300
10 2 400
19
10 2 0
10 2 100
10 2 200
10 2 300
10 2 400
20
10 2 0
10 2 100
10 2 200
10 2 300
10 2 400
0
20 2 0
20 2 100
20 2 200
20 2 300
20 2 400